In [1]:
# 生成 0 - 4 的数据集合

In [3]:
import pandas as pd

label = pd.read_csv("./mnist/train/labels.txt",header = None)
label[:5].to_csv("./mnist/train/labels4.txt",index = False,header = None)
train = pd.read_csv("./mnist/train/train.txt",header = None,sep = ' ')
train[train.iloc[:,1].isin([0,1,2,3,4])].to_csv("./mnist/train/train4.txt",index = False,header = None,sep = ' ')


label = pd.read_csv("./mnist/test/labels.txt",header = None)
label[:5].to_csv("./mnist/test/labels4.txt",index = False,header = None)
test = pd.read_csv("./mnist/test/test.txt",header = None,sep = ' ')
test[test.iloc[:,1].isin([0,1,2,3,4])].to_csv("./mnist/test/test4.txt",index = False,header = None, sep = ' ')



In [25]:
# 生成 5 类的prototxt


#### 编写一个函数，生成配置文件prototxt
#### 注意可以为每一层添加name 信息
#### 在fine - tune 时很重要 ， 一定需要把最后几层的name 改了 
#### 因为 caffe 会 默认把 名字相同的 layer的值从 caffemodel 中复制过来， 如果只是改参数而不改名，复制过程就会报错


In [ ]:
import caffe
from caffe import layers as L,params as P,proto,to_proto

#设定文件的保存路径
root='C:/Users/an/Documents/study/caffe_tutorial/caffe_tutorial/'                           #根目录
train_list=root+'mnist/train/train4.txt'     #训练图片列表
test_list=root+'mnist/test/test4.txt'        #测试图片列表
train_proto=root+'mnist/train4.prototxt'     #训练配置文件
test_proto=root+'mnist/test4.prototxt'       #测试配置文件
solver_proto=root+'mnist/solver4.prototxt'   #参数文件

def Lenet(img_list,batch_size,include_acc=False):
    #第一层，数据输入层，以ImageData格式输入
    data, label = L.ImageData(source=img_list, batch_size=batch_size, ntop=2,root_folder=root,
        transform_param=dict(scale= 0.00390625))
    #第二层：卷积层
    conv1=L.Convolution(data, kernel_size=5, stride=1,num_output=20, pad=0,weight_filler=dict(type='xavier'))
    #池化层
    pool1=L.Pooling(conv1, pool=P.Pooling.MAX, kernel_size=2, stride=2)
    #卷积层
    conv2=L.Convolution(pool1, kernel_size=5, stride=1,num_output=50, pad=0,weight_filler=dict(type='xavier'))
    #池化层
    pool2=L.Pooling(conv2, pool=P.Pooling.MAX, kernel_size=2, stride=2)
    #全连接层
    fc3=L.InnerProduct(pool2, num_output=500,weight_filler=dict(type='xavier'))
    #激活函数层
    relu3=L.ReLU(fc3, in_place=True)
    #全连接层
    fc4 = L.InnerProduct(relu3, num_output=4,weight_filler=dict(type='xavier'),name = 'fc4')
    #softmax层
    loss = L.SoftmaxWithLoss(fc4, label , name = 'loss')
    
    if include_acc:             # test阶段需要有accuracy层
        acc = L.Accuracy(fc4, label , name = 'acc')
        return to_proto(loss, acc)
    else:
        return to_proto(loss)
    
def write_net():
    #写入train.prototxt
    with open(train_proto, 'w') as f:
        f.write(str(Lenet(train_list,batch_size=64)))

    #写入test.prototxt    
    with open(test_proto, 'w') as f:
        f.write(str(Lenet(test_list,batch_size=100, include_acc=True)))
def gen_solver(solver_file,train_net,test_net):
    s=proto.caffe_pb2.SolverParameter()
    s.train_net =train_net
    s.test_net.append(test_net)
    s.test_interval = 938    #60000/64，测试间隔参数：训练完一次所有的图片，进行一次测试  
    s.test_iter.append(100)  #10000/100 测试迭代次数，需要迭代100次，才完成一次所有数据的测试
    s.max_iter = 9380       #10 epochs , 938*10，最大训练次数
    s.base_lr = 0.01    #基础学习率
    s.momentum = 0.9    #动量
    s.weight_decay = 5e-4  #权值衰减项
    s.lr_policy = 'step'   #学习率变化规则
    s.stepsize=3000         #学习率变化频率
    s.gamma = 0.1          #学习率变化指数
    s.display = 20         #屏幕显示间隔
#     s.snapshot = 938       #保存caffemodel的间隔
    s.snapshot = 2000
    s.snapshot_prefix =root+'mnist/lenet'   #caffemodel前缀
    s.type ='SGD'         #优化算法
    s.solver_mode = proto.caffe_pb2.SolverParameter.GPU    #加速
    #写入solver.prototxt
    with open(solver_file, 'w') as f:
        f.write(str(s))
write_net()
gen_solver(solver_proto,train_proto,test_proto) 

In [ ]:
# load pretrained model for train

In [ ]:
import caffe   
caffe.set_device(0)  
caffe.set_mode_gpu()
# solver = caffe.SGDSolver('C:/Users/an/Documents/study/caffe_tutorial/caffe_tutorial/mnist/solver4.prototxt')
solver = caffe.get_solver(solver_proto)


In [2]:
solver_proto

'C:/Users/an/Documents/study/caffe_tutorial/caffe_tutorial/mnist/solver4.prototxt'

In [ ]:
solver.net.copy_from("lenet_iter_8000.caffemodel")